<a href="https://colab.research.google.com/github/Khal-Ramaus/h8dsft_SQL/blob/master/h8dsft_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Haikal Usman**

NON GRADED CHALLENGE 3



*   Akses dataframe dari kaggle
*   Koneksi python dengan SQL Database menggunakan SQLite
*   Simpan masing-masing data ke masing-masing table kedalam SQLite database menggunakan keyword CREATE dan INSERT
*   Buat query untuk kolom: id_student, region, code_module, assessment_type, dan score dengan code_presentation 2014B
*   Simpan hasilnya ke pandas dataframe lalu tampilkan hasilnya kemudian simpan dalam bentuk excel

# Import library yang akan dipakai yaitu sqlite3 dan pandas

In [ ]:
import sqlite3
import pandas as pd

# Upload API kaggle lalu download dataframe dari kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haikalramadhanusman","key":"cefd766b972f55aed9f6c178a3d98dad"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d anlgrbz/student-demographics-online-education-dataoulad

 97% 41.0M/42.2M [00:01<00:00, 17.1MB/s]
100% 42.2M/42.2M [00:01<00:00, 28.2MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "student-demographics-online-education-dataoulad.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


# Menyimpan masing-masing dataframe ke variable.
Untuk dataframe vle, cell nya beda karena sempat salah memberi nama variable. Jadi vle dirun sendiri.

In [ ]:
asm = pd.read_csv('assessments.csv')
crs = pd.read_csv('courses.csv')
sAsm = pd.read_csv('studentAssessment.csv')
sInf = pd.read_csv('studentInfo.csv')
sReg = pd.read_csv('studentRegistration.csv')


In [ ]:
vle1 = pd.read_csv('vle.csv')

# Mengkoneksikan ke sqlite database

In [ ]:
def create_connection(path):
  connection=sqlite3.connect(path)
  return(connection)

connection = create_connection("NGC.sqlite")

# Membuat function untuk CREATE, INSERT dan SELECT

In [ ]:
# non select
def execute_command(conn, query):
  "Membuat fungsi untuk CREATE dan INSERT"
  cur = conn.cursor()
  cur.execute(query)
  conn.commit()

In [ ]:
def select_query(conn,query):
  "Membuat fungsi untuk query"
  cur = conn.cursor()
  cur.execute(query)
  rows = cur.fetchall()

  for row in rows:
    print(row)

# CREATE dan INSERT table

## Create dan insert table assessment

In [ ]:
assessments='''
CREATE TABLE assessments (
  code_module varchar(3),
  code_presentation varchar(5),
  id_assessment INTEGER,
  assessment_type varchar(10),
  date INTEGER,
  weight FLOAT,
  FOREIGN KEY (code_module, code_presentation) REFERENCES courses(code_module, code_presentation)
)
'''
execute_command(connection, assessments)

Cek null value menggunakan syntax:

In [ ]:
asm.isnull().values.any()

True

In [ ]:
asm.head(10)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,NaN,100.0
6,AAA,2014J,1758,TMA,19.0,10.0
7,AAA,2014J,1759,TMA,54.0,20.0
8,AAA,2014J,1760,TMA,117.0,20.0
9,AAA,2014J,1761,TMA,166.0,20.0


Mengganti null value dengan 0:

In [ ]:
asm['date'].fillna(0, inplace=True)

In [ ]:
asm.head(10)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,0.0,100.0
6,AAA,2014J,1758,TMA,19.0,10.0
7,AAA,2014J,1759,TMA,54.0,20.0
8,AAA,2014J,1760,TMA,117.0,20.0
9,AAA,2014J,1761,TMA,166.0,20.0


Insert semua yang ada di daframe assessments ke dalam table assessment menggunakan looping:

In [ ]:
for i in range(len(asm)):
  inAsm = '''
  INSERT INTO assessments (code_module,code_presentation,id_assessment,assessment_type,date,weight)
  VALUES ('{}','{}',{},'{}',{},{})
  '''.format(asm['code_module'][i],asm['code_presentation'][i],asm['id_assessment'][i],asm['assessment_type'][i],asm['date'][i],asm['weight'][i])
  execute_command(connection, inAsm)

In [ ]:
select_query(connection, "SELECT * FROM assessments")

('AAA', '2013J', 1752, 'TMA', 19, 10.0)
('AAA', '2013J', 1753, 'TMA', 54, 20.0)
('AAA', '2013J', 1754, 'TMA', 117, 20.0)
('AAA', '2013J', 1755, 'TMA', 166, 20.0)
('AAA', '2013J', 1756, 'TMA', 215, 30.0)
('AAA', '2013J', 1757, 'Exam', 0, 100.0)
('AAA', '2014J', 1758, 'TMA', 19, 10.0)
('AAA', '2014J', 1759, 'TMA', 54, 20.0)
('AAA', '2014J', 1760, 'TMA', 117, 20.0)
('AAA', '2014J', 1761, 'TMA', 166, 20.0)
('AAA', '2014J', 1762, 'TMA', 215, 30.0)
('AAA', '2014J', 1763, 'Exam', 0, 100.0)
('BBB', '2013B', 14991, 'CMA', 54, 1.0)
('BBB', '2013B', 14992, 'CMA', 89, 1.0)
('BBB', '2013B', 14993, 'CMA', 124, 1.0)
('BBB', '2013B', 14994, 'CMA', 159, 1.0)
('BBB', '2013B', 14995, 'CMA', 187, 1.0)
('BBB', '2013B', 14984, 'TMA', 19, 5.0)
('BBB', '2013B', 14985, 'TMA', 47, 18.0)
('BBB', '2013B', 14986, 'TMA', 89, 18.0)
('BBB', '2013B', 14987, 'TMA', 124, 18.0)
('BBB', '2013B', 14988, 'TMA', 159, 18.0)
('BBB', '2013B', 14989, 'TMA', 187, 18.0)
('BBB', '2013B', 14990, 'Exam', 0, 100.0)
('BBB', '2013J', 15

## Create dan insert table courses

In [ ]:
courses='''
CREATE TABLE courses (
  code_module varchar(3),
  code_presentation varchar(5),
  module_presentation_length INTEGER
)
'''
execute_command(connection, courses)

In [ ]:
for i in range(len(crs)):
  inAsm = '''
  INSERT INTO courses (code_module,code_presentation,module_presentation_length)
  VALUES ('{}','{}',{})
  '''.format(crs['code_module'][i],crs['code_presentation'][i],crs['module_presentation_length'][i])
  execute_command(connection, inAsm)

In [ ]:
select_query(connection, "SELECT * From courses")

('AAA', '2013J', 268)
('AAA', '2014J', 269)
('BBB', '2013J', 268)
('BBB', '2014J', 262)
('BBB', '2013B', 240)
('BBB', '2014B', 234)
('CCC', '2014J', 269)
('CCC', '2014B', 241)
('DDD', '2013J', 261)
('DDD', '2014J', 262)
('DDD', '2013B', 240)
('DDD', '2014B', 241)
('EEE', '2013J', 268)
('EEE', '2014J', 269)
('EEE', '2014B', 241)
('FFF', '2013J', 268)
('FFF', '2014J', 269)
('FFF', '2013B', 240)
('FFF', '2014B', 241)
('GGG', '2013J', 261)
('GGG', '2014J', 269)
('GGG', '2014B', 241)


## Create dan insert table studentAssessment

In [ ]:
studentAssessment='''
CREATE TABLE studentAssessment (
  id_assessment INTEGER,
  id_student INTEGER,
  date_submitted INTEGER,
  score INTEGER,
  FOREIGN KEY(id_student) REFERENCES studentInfo(id_student),
  FOREIGN KEY(id_assessment) REFERENCES assessments(id_assessment)
)
'''
execute_command(connection, studentAssessment)

Cek null value menggunakan syntax:

In [ ]:
sAsm.isnull().values.any()

True

In [ ]:
sAsm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  int64  
 1   id_student      173912 non-null  int64  
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB


Mengganti null value dengan 0:

In [ ]:
sAsm['score'].fillna(0, inplace=True) # jika score kosong artinya tidka ada nilai, maka diisi 0 aja

In [ ]:
sAsm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  int64  
 1   id_student      173912 non-null  int64  
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173912 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.6 MB


In [ ]:
for i in range(len(sAsm)):
  input = '''
  INSERT INTO studentAssessment (id_assessment,id_student,date_submitted,score)
  VALUES ({},{},{},{})
  '''.format(sAsm['id_assessment'][i],sAsm['id_student'][i],sAsm['date_submitted'][i],sAsm['score'][i])
  execute_command(connection, input)

In [ ]:
select_query(connection, "SELECT * From studentAssessment WHERE studentAssessment.id_assessment=1752")

(1752, 11391, 18, 78)
(1752, 28400, 22, 70)
(1752, 31604, 17, 72)
(1752, 32885, 26, 69)
(1752, 38053, 19, 79)
(1752, 45462, 20, 70)
(1752, 45642, 18, 72)
(1752, 52130, 19, 72)
(1752, 53025, 9, 71)
(1752, 57506, 18, 68)
(1752, 58873, 19, 73)
(1752, 59185, 18, 67)
(1752, 62155, 17, 73)
(1752, 63400, 19, 83)
(1752, 65002, 17, 66)
(1752, 70464, 19, 59)
(1752, 71361, 19, 82)
(1752, 74372, 22, 60)
(1752, 75091, 18, 67)
(1752, 77367, 18, 73)
(1752, 91265, 21, 75)
(1752, 94961, 17, 74)
(1752, 98094, 18, 62)
(1752, 100893, 17, 63)
(1752, 101781, 16, 84)
(1752, 102806, 19, 80)
(1752, 102952, 19, 76)
(1752, 104476, 30, 85)
(1752, 106247, 32, 67)
(1752, 106577, 18, 57)
(1752, 110175, 18, 66)
(1752, 111717, 10, 80)
(1752, 113295, 19, 81)
(1752, 114017, 18, 66)
(1752, 114999, 18, 74)
(1752, 116541, 19, 76)
(1752, 116692, 16, 72)
(1752, 118983, 19, 73)
(1752, 123044, 25, 75)
(1752, 127582, 26, 69)
(1752, 129955, 18, 85)
(1752, 132976, 18, 63)
(1752, 134143, 18, 87)
(1752, 135400, 19, 72)
(1752, 13787

## Create dan insert table studentInfo

In [ ]:
studentInfo='''
CREATE TABLE studentInfo (
  code_module varchar(3),
  code_presentation varchar(5),
  id_student INTEGER,
  gender varchar(1),
  region varchar(50),
  highest_education varchar(50),
  imd_band varchar(50),
  age_band varchar(50),
  num_of_prev_attempts INTEGER,
  studied_credits INTEGER,
  disability varchar(1),
  final_result varchar(50),
  FOREIGN KEY (code_module, code_presentation) REFERENCES courses(code_module, code_presentation)
)
'''
execute_command(connection, studentInfo)

In [ ]:
sInf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
 11  final_result          32593 non-null  object
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


In [ ]:
for i in range(len(sInf)):
  input = '''
  INSERT INTO studentInfo (code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result)
  VALUES ('{}','{}',{},'{}','{}','{}','{}','{}',{},{},'{}','{}')
  '''.format(sInf['code_module'][i],sInf['code_presentation'][i],sInf['id_student'][i],sInf['gender'][i],sInf['region'][i],sInf['highest_education'][i],sInf['imd_band'][i],sInf['age_band'][i],sInf['num_of_prev_attempts'][i],sInf['studied_credits'][i],sInf['disability'][i],sInf['final_result'][i])
  execute_command(connection, input)

In [ ]:
select_query(connection, "SELECT * From studentInfo")

Streaming output truncated to the last 5000 lines.
('FFF', '2014B', 2095576, 'M', 'South East Region', 'A Level or Equivalent', '80-90%', '0-35', 0, 90, 'N', 'Pass')
('FFF', '2014B', 2097046, 'M', 'Wales', 'Lower Than A Level', '60-70%', '35-55', 0, 60, 'N', 'Fail')
('FFF', '2014B', 2114529, 'M', 'East Anglian Region', 'Lower Than A Level', '20-30%', '0-35', 0, 60, 'N', 'Withdrawn')
('FFF', '2014B', 2131008, 'M', 'London Region', 'A Level or Equivalent', '50-60%', '35-55', 0, 60, 'N', 'Pass')
('FFF', '2014B', 2156172, 'M', 'South Region', 'A Level or Equivalent', '40-50%', '35-55', 0, 60, 'N', 'Withdrawn')
('FFF', '2014B', 2161710, 'M', 'Wales', 'A Level or Equivalent', '70-80%', '35-55', 2, 60, 'N', 'Withdrawn')
('FFF', '2014B', 2164266, 'M', 'East Anglian Region', 'Lower Than A Level', '30-40%', '0-35', 3, 180, 'Y', 'Fail')
('FFF', '2014B', 2176759, 'M', 'Yorkshire Region', 'HE Qualification', '30-40%', '35-55', 0, 60, 'N', 'Pass')
('FFF', '2014B', 2186992, 'M', 'West Midlands Region

## Create dan insert table studentRegistration

In [ ]:
studentRegistration='''
CREATE TABLE studentRegistration (
  code_module varchar(3),
  code_presentation varchar(5),
  id_student INTEGER,
  date_registration INTEGER,
  date_unregistration INTEGER,
  FOREIGN KEY (code_module, code_presentation) REFERENCES courses(code_module, code_presentation),
  FOREIGN KEY(id_student) REFERENCES studentInfo(id_student)
)
'''
execute_command(connection, studentRegistration)

Cek null value menggunakan syntax:

In [ ]:
sReg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          32593 non-null  object 
 1   code_presentation    32593 non-null  object 
 2   id_student           32593 non-null  int64  
 3   date_registration    32548 non-null  float64
 4   date_unregistration  10072 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


Mengganti null value dengan 0:

In [ ]:
sReg['date_registration'].fillna(0, inplace=True)

In [ ]:
sReg['date_unregistration'].fillna(0, inplace=True)

In [ ]:
for i in range(len(sReg)):
  input = '''
  INSERT INTO studentRegistration (code_module,code_presentation,id_student,date_registration,date_unregistration)
  VALUES ('{}','{}',{},{},{})
  '''.format(sReg['code_module'][i],sReg['code_presentation'][i],sReg['id_student'][i],sReg['date_registration'][i],sReg['date_unregistration'][i])
  execute_command(connection, input)

In [ ]:
select_query(connection, "SELECT * From studentRegistration")

Streaming output truncated to the last 5000 lines.
('FFF', '2014B', 2095576, -52, 0)
('FFF', '2014B', 2097046, -44, 0)
('FFF', '2014B', 2114529, -69, 226)
('FFF', '2014B', 2131008, -22, 0)
('FFF', '2014B', 2156172, -186, 226)
('FFF', '2014B', 2161710, -23, 178)
('FFF', '2014B', 2164266, -22, 0)
('FFF', '2014B', 2176759, -28, 0)
('FFF', '2014B', 2186992, -66, 0)
('FFF', '2014B', 2209846, -21, 16)
('FFF', '2014B', 2222615, -61, 0)
('FFF', '2014B', 2238019, -132, 0)
('FFF', '2014B', 2239539, -87, -16)
('FFF', '2014B', 2242595, -23, 0)
('FFF', '2014B', 2242732, -29, 0)
('FFF', '2014B', 2244285, -128, 0)
('FFF', '2014B', 2256318, -88, 0)
('FFF', '2014B', 2257112, -169, 0)
('FFF', '2014B', 2264054, -29, 0)
('FFF', '2014B', 2265419, -129, 206)
('FFF', '2014B', 2270880, -21, 0)
('FFF', '2014B', 2298316, -42, 22)
('FFF', '2014B', 2299808, -27, 0)
('FFF', '2014B', 2303413, -34, 0)
('FFF', '2014B', 2306016, -67, 0)
('FFF', '2014B', 2322813, -109, 52)
('FFF', '2014B', 2327494, -40, 0)
('FFF', '201

## Create dan insert table vle

In [ ]:
vle='''
CREATE TABLE vle (
  id_site INTEGER PRIMARY KEY,
  code_module varchar(3),
  code_presentation varchar(5),
  activity_type varchar(50),
  week_from FLOAT,
  week_to FLOAT,
  FOREIGN KEY (code_module, code_presentation) REFERENCES courses(code_module, code_presentation)
)
'''
execute_command(connection, vle)

Cek null value menggunakan syntax:

In [ ]:
vle1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            6364 non-null   int64  
 1   code_module        6364 non-null   object 
 2   code_presentation  6364 non-null   object 
 3   activity_type      6364 non-null   object 
 4   week_from          1121 non-null   float64
 5   week_to            1121 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 298.4+ KB


Mengganti null value dengan 0:

In [ ]:
vle1['week_from'].fillna(0, inplace=True)

In [ ]:
vle1['week_to'].fillna(0, inplace=True)

In [ ]:
for i in range(len(vle1)):
  input = '''
  INSERT INTO vle (id_site,code_module,code_presentation,activity_type,week_from,week_to)
  VALUES ({},'{}','{}','{}',{},{})
  '''.format(vle1['id_site'][i],vle1['code_module'][i],vle1['code_presentation'][i],vle1['activity_type'][i],vle1['week_from'][i],vle1['week_to'][i])
  execute_command(connection, input)

In [ ]:
select_query(connection, "SELECT * From vle")

Streaming output truncated to the last 5000 lines.
(547871, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547872, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547873, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547874, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547875, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547876, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547877, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547878, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547879, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547880, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547881, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547882, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547883, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547884, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547885, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547886, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547887, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547888, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547889, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547890, 'DDD', '2013B', 'subpage', 0.0, 0.0)
(547891, 'DDD', '2013B', 'sub

# Buat query untuk kolom: id_student, region, code_module, assessment_type, dan score dengan code_presentation 2014B

In [ ]:
query='''
SELECT si.id_student,si.region,a.code_module,a.assessment_type,sa.score 
FROM studentAssessment as sa 
LEFT JOIN studentInfo as si ON sa.id_student=si.id_student
LEFT JOIN assessments as a ON sa.id_assessment=a.id_assessment
WHERE a.code_presentation='2014B'
'''
select_query(connection, query)

Streaming output truncated to the last 5000 lines.
(595343, 'Yorkshire Region', 'GGG', 'TMA', 70)
(595572, 'South Region', 'GGG', 'TMA', 80)
(596382, 'North Western Region', 'GGG', 'TMA', 65)
(596513, 'Yorkshire Region', 'GGG', 'TMA', 72)
(596513, 'Yorkshire Region', 'GGG', 'TMA', 72)
(596588, 'East Midlands Region', 'GGG', 'TMA', 90)
(596751, 'South West Region', 'GGG', 'TMA', 60)
(597010, 'South Region', 'GGG', 'TMA', 82)
(597037, 'North Western Region', 'GGG', 'TMA', 65)
(598017, 'North Western Region', 'GGG', 'TMA', 81)
(598127, 'North Western Region', 'GGG', 'TMA', 50)
(598496, 'South East Region', 'GGG', 'TMA', 60)
(598564, 'South Region', 'GGG', 'TMA', 60)
(599144, 'East Midlands Region', 'GGG', 'TMA', 54)
(599276, 'East Anglian Region', 'GGG', 'TMA', 85)
(599467, 'South West Region', 'GGG', 'TMA', 60)
(599620, 'West Midlands Region', 'GGG', 'TMA', 90)
(600035, 'West Midlands Region', 'GGG', 'TMA', 64)
(600714, 'North Western Region', 'GGG', 'TMA', 78)
(601064, 'East Anglian Reg

# Query menggunakan pandas

In [ ]:
data=pd.read_sql_query(query, connection)
data

,id_student,region,code_module,assessment_type,score
0,558921,West Midlands Region,BBB,TMA,86
1,559090,South East Region,BBB,TMA,54
2,559336,West Midlands Region,BBB,TMA,74
3,559381,South Region,BBB,TMA,63
4,559467,East Anglian Region,BBB,TMA,77
...,...,...,...,...,...
48949,635035,West Midlands Region,GGG,CMA,100
48950,635052,North Western Region,GGG,CMA,80
48951,635233,North Western Region,GGG,CMA,100
48952,635506,London Region,GGG,CMA,60


In [ ]:
data.to_excel('NGC.xlsx')